In [7]:
import xgboost as xgb
from xgboost import XGBClassifier

model = xgb.Booster()
model.load_model('/home/ai_car/AthenaBow/model/1/model.json')

model.predict(xgb.DMatrix())

ValueError: training data did not have the following fields: AGREEMENTID, PRODUCT, NC, DayofWeek, TimeCall

In [9]:
import pandas as pd
from tqdm import tqdm
from datetime import datetime, date
import pickle
from time import time
from src.utils import libSort
import calendar
from calendar import monthrange
from src.utils.QueryData import GetIndex, checkNumberDayRemaining, SearchInfor
from src.utils.mergeToDataFrame import format_time
from src.utils.convert_data_type import convertList

IDUser = str(input("Please type Agreedment ID: "))
dataBase = pd.read_csv('/home/ai_car/AthenaBow/data/TotalData.csv')
start = time()
listID = GetIndex(data= dataBase, id= IDUser)
if len(listID) == 0:
    raise Exception("DataNotFound: The data which you just fill in not in Database")
elif len(listID) > 1:
    Phone, Product, Nc = SearchInfor(data= dataBase, index= listID[0], NCIndex= listID[-1])
    NowDay, LimitInMonth, NowMonth = checkNumberDayRemaining()
    xgbModel = pickle.load(open('/home/ai_car/AthenaBow/model/24-12/xgb.sav', 'rb'))
    whatInsideToModel = [[Phone, Product, Nc]]
    listResult = pd.DataFrame()
    for i in tqdm(range(len(whatInsideToModel))):
        for j in range(NowDay, LimitInMonth + 1):
            perCent = []
            listTimeAvailabel = []
            for k in tqdm(range(480, 12001, 30)):
                preds = xgbModel.predict_proba([[whatInsideToModel[i][0],
                                                whatInsideToModel[i][1],
                                                whatInsideToModel[i][2],
                                                k,
                                                j,
                                                NowMonth]])
                preds = preds.tolist()
                perCent.append(preds[0][1])
                listTimeAvailabel.append(k)
            whenIsAGoodTime = libSort.nlargest(3,zip(perCent, listTimeAvailabel))
            whenIsAGoodTime = convertList(whenIsAGoodTime)
            StandardScaleTime = []
            for ff in range(len(whenIsAGoodTime)):
                HereWeLive = [int(whenIsAGoodTime[ff][0] * 100), int(whenIsAGoodTime[ff][1])]
                StandardScaleTime.append(HereWeLive)
            DayinWeeks = date(year=datetime.date(datetime.now()).year,
                              month=NowMonth,
                              day= int(j))
            listResult[f'{j}/{NowMonth}/{datetime.date(datetime.now()).year} - {calendar.day_name[DayinWeeks.weekday()]}'] = StandardScaleTime
end = time()
print(end - start)
listResult


/home/ai_car/.local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3553: DtypeWarning: Columns (8,9,13,14,15,17,18,25,36,37,40,43,48) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
100%|██████████| 1/1 [00:03<00:00,  3.17s/it]

20.278660535812378


,26/12/2022 - Monday,27/12/2022 - Tuesday,28/12/2022 - Wednesday,29/12/2022 - Thursday,30/12/2022 - Friday,31/12/2022 - Saturday
0,"[90, 570]","[90, 630]","[89, 630]","[90, 570]","[90, 630]","[90, 630]"
1,"[89, 600]","[89, 570]","[89, 570]","[90, 630]","[90, 660]","[90, 570]"
2,"[89, 630]","[89, 600]","[89, 540]","[90, 540]","[89, 600]","[89, 540]"


In [14]:
HereWeLive

('98', 555)

In [2]:
for m in listResult.columns:
    print(m)

26/12/2022 - Monday
27/12/2022 - Tuesday
28/12/2022 - Wednesday
29/12/2022 - Thursday
30/12/2022 - Friday
31/12/2022 - Saturday


In [3]:
int(m[:2])

31

In [7]:
extractDay = int(m.split('/')[0])
extractDay

31

In [17]:
j.tolist()

[[92, 555],
 [96, 555],
 [98, 555],
 [98, 555],
 [98, 555],
 [98, 555],
 [98, 615],
 [98, 555]]

: 

In [4]:
import pandas as pd
from tqdm import tqdm
from datetime import datetime
import pickle
from time import time
from src.utils import libSort
from calendar import monthrange

start = time()
a = str(input("Please paste the path of file: "))
file = pd.read_excel(a)
days = datetime.now()
days = int(days.weekday()) + 2
xgb_reg = pickle.load(open("/home/ai_car/AthenaBow/model/xgb_reg2.sav", "rb"))
listTimeAvailabel = [6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23]
listResult = []
for i in tqdm(range(len(file.columns[0]))):
    for j in range(len(listTimeAvailabel)):
        perCent = []
        preds = xgb_reg.predict_proba([[file[file.columns[0]][i],
                                        file[file.columns[1]][i],
                                        file[file.columns[2]][i],
                                        listTimeAvailabel[j],
                                        days]])
        preds = preds.tolist()
        perCent.append(preds[0][1])
    GoodTime = libSort.nlargest(3,zip(perCent, listTimeAvailabel))
    listResult.append(GoodTime)
time() - start
            

In [4]:
import pandas as pd
from tqdm import tqdm
from datetime import datetime
import pickle
from src.utils.encodeData import encodeProductInference
from time import time
from src.utils import libSort

start = time()
# data_exam = [[3216305, 7, 5985332],[3523308, 9, 5985112],[3216308, 2, 5365334],[3216417, 1, 6315334],[3231208, 9, 5983134]]
a = int(input("Please input your id: "))
b = str(input("Please input your Product: "))
b = encodeProductInference(b)
c = int(input("Please input your NC: "))
data_exam = [[a,b,c]]
xgb_reg = pickle.load(open("/home/ai_car/AthenaBow/model/xgb_NODay.sav", "rb"))
listResult = []
for i in tqdm(range(len(data_exam))):
    perCent = []
    listTimeAvailabel = []
    for j in range(480, 12001, 15):
        preds = xgb_reg.predict_proba([[data_exam[i][0],
                                        data_exam[i][1],
                                        data_exam[i][2],
                                        j]])
        preds = preds.tolist()
        perCent.append(preds[0][1])
        listTimeAvailabel.append(j)
    GoodTime = libSort.nlargest(5,zip(perCent, listTimeAvailabel))
    listResult.append(GoodTime)
time() - start

ValueError: invalid literal for int() with base 10: ''

In [5]:
a = a + (10,)
a

(4, 10)

In [2]:
GoodTime

[(0.8860887885093689, 945),
 (0.8860887885093689, 930),
 (0.8860887885093689, 915),
 (0.8860887885093689, 900),
 (0.6602908372879028, 885)]

In [3]:
listResult

[[(0.8860887885093689, 945),
  (0.8860887885093689, 930),
  (0.8860887885093689, 915),
  (0.8860887885093689, 900),
  (0.6602908372879028, 885)]]

In [ ]:
listResult

In [9]:
from src.utils.mergeToDataFrame import format_time
newTime = []
for i in tqdm(range(len(listResult))):
    charac = []
    for j in range(len(listResult[i])):
        time = format_time(listResult[i][j][1])
        charac.append(time)
    newTime.append(charac)

100%|██████████| 1/1 [00:00<00:00, 11275.01it/s]


In [8]:
newTime

[['15 h 45 ', '15 h 30 ', '15 h 15 ', '15 h 0 ', '14 h 45 ']]

In [ ]:
ngày - thứ - giờ - (15m) accuracy

In [8]:
data_exam = [[3216305, 7, 5985332],[3523308, 9, 5985112],[3216308, 2, 5365334],[3216417, 1, 6315334],[3231208, 9, 5983134]]
data_exam[0][1]

7

In [8]:
import numpy as np


listTimeAvailabel = [6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23]
lamda = sorted((e,i) for i,e in enumerate(preds))
# for index, item in enumerate(listTimeAvailabel):
#     print(index)
lamda[0]

([1, 3], 0)

In [9]:
lamda[0][0][1]

3

In [41]:
import numpy as np
kapa = [123,3123]
lata = np.array(kapa, dtype=float)
# list(lata)

array([ 123., 3123.])

In [22]:
import pickle
xgb_reg = pickle.load(open("/home/ai_car/AthenaBow/model/xgb_reg2.sav", "rb"))
preds = xgb_reg.predict_proba([[901327305, 0, 5985334, 7, 16]])
xgb_reg.set_params()

AttributeError: 'XGBClassifier' object has no attribute 'set_param'

In [21]:
preds.tolist()

[[0.6256044507026672, 0.37439554929733276]]

In [28]:
file = pd.read_excel('/home/ai_car/AthenaBow/data/import mafc xlsx/debt_reminder_202200707.xlsx')

In [29]:
file.columns[0]

'CUSTOMER'

In [33]:
a = file[file.columns[0]]
a

0           NGUYỄN THẾ NGHI
1               LƯƠNG  TIẾT
2       BẠCH VŨ KIỀU PHƯƠNG
3        BÙI THỊ ĐOAN TRINH
4           NGUYỄN ĐỨC NGỌC
               ...         
9177        DANH THỊ THƯƠNG
9178         NGUYỄN VĂN MÃO
9179            MAI VĂN SƠN
9180        NGÔ THỊ LIỄU CA
9181         PHẠM MINH ĐƯỢC
Name: CUSTOMER, Length: 9182, dtype: object